## DigiSafe
---

In [2]:
:cache 1000
:dep apache-avro
:dep argon2
:dep base64ct = { features = ["alloc"] }
:dep chacha20poly1305
:dep getrandom
:dep lz4
:dep reed-solomon
:dep serde = { features = ["derive"] }
:dep sha3

cache: 1000 MiB


In [3]:
pub mod Database {

    // Database<Unlocked> <-> Avro <-> LZ4 <-> ChaCha20Poly1305 <-> Base64 <-> Database<Locked>
    // Database<Locked> -> Avro -> Reed-Solomon -> Vec<u8> -> name.digisafe

    use serde::{Deserialize, Serialize};
    use std::collections::HashMap;
    use std::io::{Read, Write};

    pub fn hash_lite(msg: &Vec<u8>) -> Vec<u8> {
        use sha3::{Digest, Sha3_256};
        Sha3_256::digest(Sha3_256::digest(msg)).to_vec()
    }

    pub fn hash_heavy(msg: String) -> Vec<u8> {
        use argon2::{Algorithm, Argon2, ParamsBuilder, password_hash::{PasswordHasher, SaltString}, Version};
        let lite_hash = hash_lite(&msg.into());
        let params = ParamsBuilder::new().m_cost(2u32.pow(20)).t_cost(2).p_cost(8).output_len(32).build().unwrap();
        let hasher = Argon2::new(Algorithm::Argon2id, Version::V0x13, params);
        let salt = SaltString::encode_b64(b"digisafe").unwrap();
        let heavy_hash = hasher.hash_password(&lite_hash, &salt).unwrap().hash.unwrap().as_bytes().to_owned();
        hash_lite(&heavy_hash)
    }

    pub fn to_base64(msg: &Vec<u8>) -> String {
        use base64ct::{Base64, Encoding};
        Base64::encode_string(msg)
    }

    pub fn from_base64(msg_enc: &String) -> Vec<u8> {
        use base64ct::{Base64, Encoding};
        Base64::decode_vec(&msg_enc).unwrap()
    }

    pub fn compress(msg: Vec<u8>) -> Vec<u8> {
        use lz4::{Decoder, EncoderBuilder};
        let mut encoder = EncoderBuilder::new().level(9).build(vec![]).unwrap();
        encoder.write(&msg[..]);
        encoder.finish().0
    }

    pub fn decompress(msg_enc: Vec<u8>) -> Vec<u8> {
        use lz4::Decoder;
        let mut msg = vec![];
        {
            let mut decoder = Decoder::new(&msg_enc[..]).unwrap();
            std::io::copy(&mut decoder, &mut msg);
        }
        msg
    }
    
    pub fn encrypt(msg: Vec<u8>, key: Vec<u8>, nonce: Vec<u8>) -> Vec<u8> {
        use chacha20poly1305::{aead::{Aead, KeyInit}, XChaCha20Poly1305};
        let key: [u8; 32] = key.try_into().unwrap();
        let nonce: [u8; 24] = nonce.try_into().unwrap();
        let cipher = XChaCha20Poly1305::new(&key.into());
        cipher.encrypt(&nonce.into(), &msg[..]).unwrap()
    }

    pub fn decrypt(msg_enc: Vec<u8>, key: Vec<u8>, nonce: Vec<u8>) -> Option<Vec<u8>> {
        use chacha20poly1305::{aead::{Aead, KeyInit}, XChaCha20Poly1305};
        let key: [u8; 32] = key.try_into().unwrap();
        let nonce: [u8; 24] = nonce.try_into().unwrap();
        let cipher = XChaCha20Poly1305::new(&key.into());
        if let Ok(msg) = cipher.decrypt(&nonce.into(), msg_enc.as_ref()) {
            Some(msg)
        } else {
            None
        }
    }

    fn to_ecc_batch(dat: Vec<u8>) -> Vec<u8> {
        use reed_solomon::Encoder;
        let enc = Encoder::new(8);
        Vec::from(&enc.encode(&dat[..])[..])
    }

    fn from_ecc_batch(dat_enc: Vec<u8>) -> Vec<u8> {
        use reed_solomon::Decoder;
        let dec = Decoder::new(8);
        dec.correct(&dat_enc[..], None).unwrap().data().to_owned()
    }

    fn to_ecc(dat: Vec<u8>) -> Vec<u8> {
        let dat_len = dat.len();
        let batch_size = 247usize;
        let batches = ((dat_len as f64)/(batch_size as f64)).ceil() as usize;
        let mut dat_enc = Vec::<u8>::with_capacity(dat_len + 8*batches);
        let mut a = 0usize;
        for _ in 0..batches {
            let b = std::cmp::min(a + batch_size, dat_len);
            let dat_slice = &dat[a..b];
            let enc_dat_slice = to_ecc_batch(dat_slice.to_vec());
            dat_enc.extend_from_slice(&enc_dat_slice[..]);
            a = std::cmp::min(a + batch_size, dat_len);
        }
        dat_enc
    }

    fn from_ecc(dat_enc: Vec<u8>) -> Vec<u8> {
        let dat_enc_len = dat_enc.len();
        let batch_size = 255usize;
        let batches = ((dat_enc_len as f64)/(batch_size as f64)).ceil() as usize;
        let mut dat = Vec::<u8>::with_capacity(dat_enc_len - 8*batches);
        let mut a = 0usize;
        for _ in 0..batches {
            let b = std::cmp::min(a + batch_size, dat_enc_len);
            let dat_enc_slice = &dat_enc[a..b];
            let dat_slice = from_ecc_batch(dat_enc_slice.to_vec());
            dat.extend_from_slice(&dat_slice[..]);
            a += batch_size;
            a = std::cmp::min(a, dat_enc_len);
        }
        dat
    }

    const unlocked_schema_raw: &str = r#"
    {
        "type": "record",
        "name": "unlocked",
        "fields": [
            {"name": "db", "type": {"type": "map", "values": "string"}},
            {"name": "meta", "type": {"type": "map", "values": "string"}}
        ]
    }
    "#;
    
    const locked_schema_raw: &str = r#"
    {
        "type": "record",
        "name": "locked",
        "fields": [
            {"name": "db", "type": "string"},
            {"name": "meta", "type": {"type": "map", "values": "string"}}
        ]
    }
    "#;

    #[derive(Debug, Deserialize, Serialize)]
    pub struct AvroUnlocked {
        db: HashMap<String, String>,
        meta: HashMap<String, String>,
    }

    #[derive(Debug, Deserialize, Serialize)]
    pub struct AvroLocked {
        db: String,
        meta: HashMap<String, String>,
    }

    #[derive(Default, Debug)]
    pub struct Unlocked {
        db: HashMap<String, String>,
        meta: HashMap<String, String>,
        key: Vec<u8>,
    }

    #[derive(Debug, Default)]
    pub struct Locked {
        db: String,
        meta: HashMap<String, String>,
        key: Vec<u8>,
    }

    impl AvroUnlocked {
        pub fn to_unlocked_struct(self, key: Vec<u8>) -> Unlocked {
             Unlocked {
                 db: self.db,
                 meta: self.meta,
                 key: key,
             }
        }
    }

    impl AvroLocked {
        pub fn to_locked_struct(self) -> Locked {
             Locked {
                 db: self.db,
                 meta: self.meta,
                 key: Default::default(),
             }
        }
    }
    
    impl Unlocked {
        
        pub fn new(key: impl Into<String>) -> Self {
            let mut db = Unlocked {
                db: Default::default(),
                meta: Default::default(),
                key: hash_heavy(key.into()),
            };
            let ts = std::time::SystemTime::now()
                     .duration_since(std::time::UNIX_EPOCH).unwrap().as_secs().to_string();
            let mut randy = [0u8; 32];
            getrandom::getrandom(&mut randy);
            let randy = to_base64(&randy.to_vec());
            db.set_meta("uid", randy);
            db.set_meta("app", "digisafe");
            db.set_meta("revision", "0");
            db.set_meta("version", "1.0.0");
            db.set_meta("timestamp", ts);
            db
        }
        
        pub fn lock(self) -> Locked {
            let cadb = compress(self.to_avro());
            let nonce = hash_lite(&cadb)[..24].to_vec();
            let mut meta = HashMap::<String, String>::new();
            meta.extend(self.meta);
            meta.insert("nonce".into(), to_base64(&nonce));
            let db = to_base64(&encrypt(cadb, self.key.clone(), nonce));
            Locked {
                db: db,
                meta: meta,
                key: self.key,
            }
        }
    
        pub fn get(&self, akey: &String) -> Option<String> {
            self.db.get(akey).cloned()
        }
    
        
        pub fn set(&mut self, akey: impl Into<String>, aval: impl Into<String>) {
            self.db.insert(akey.into(), aval.into());
        }
    
        pub fn get_private(&self, akey: &String) -> Option<String> {
            self.db.get(&("_".to_string() + &akey)).cloned()
        }
    
        
        pub fn set_private(&mut self, akey: impl Into<String>, aval: impl Into<String>) {
            self.db.insert("_".to_string() + &akey.into(), aval.into());
        }
    
        pub fn get_meta(&self, akey: &String) -> Option<String> {
            self.meta.get(akey).cloned()
        }
    
        
        pub fn set_meta(&mut self, akey: impl Into<String>, aval: impl Into<String>) {
            self.meta.insert(akey.into(), aval.into());
        }

        pub fn merge(&mut self, other: Unlocked) {
            self.db.extend(other.db);
        }

        pub fn to_avro_struct(&self) -> AvroUnlocked {
            AvroUnlocked {
                db: self.db.clone(),
                meta: self.meta.clone(),
            }
        }

        pub fn to_avro(&self) -> Vec<u8> {
            use apache_avro::{to_value, Schema, Writer};
            let schema = Schema::parse_str(unlocked_schema_raw).unwrap();
            let mut writer = Writer::new(&schema, Vec::new());
            writer.append(to_value(&self.to_avro_struct()).unwrap()).unwrap();
            let avro_dat = writer.into_inner().unwrap();
            avro_dat
        }

        pub fn from_avro(avro_dat: Vec<u8>, key: Vec<u8>) -> Unlocked {
            use apache_avro::{from_value, Reader};
            let reader = Reader::new(&avro_dat[..]).unwrap();
            let db = from_value::<AvroUnlocked>(&reader.last().unwrap().unwrap()).unwrap();
            db.to_unlocked_struct(key)
        }
    }

    /*
    impl std::fmt::Debug for Unlocked {
        fn fmt(&self, objf: &mut std::fmt::Formatter) -> std::fmt::Result {
            write!(objf, "Database::Unlocked: {{ redacted }}")
        }
    }

    impl std::fmt::Debug for AvroUnlocked {
        fn fmt(&self, objf: &mut std::fmt::Formatter) -> std::fmt::Result {
            write!(objf, "Database::AvroUnlocked: {{ redacted }}")
        }
    }
    */

    impl Locked {
        
        pub fn unlock(self, password: impl Into<String>) -> Option<Unlocked> {
            let key = hash_heavy(password.into());
            let nonce = from_base64(&self.meta.get("nonce").unwrap());
            let cradb = decrypt(from_base64(&self.db), key.clone(), nonce);
            if cradb.is_none() {
                return None;
            }
            let radb = decompress(cradb.unwrap());
            let db = Unlocked::from_avro(radb, key);
            Some(db)
        }

        pub fn to_avro_struct(&self) -> AvroLocked {
            AvroLocked {
                db: self.db.clone(),
                meta: self.meta.clone(),
            }
        }

        pub fn to_avro(&self) -> Vec<u8> {
            use apache_avro::{to_value, Schema, Writer};
            let schema = Schema::parse_str(locked_schema_raw).unwrap();
            let mut writer = Writer::new(&schema, Vec::new());
            writer.append(to_value(&self.to_avro_struct()).unwrap()).unwrap();
            let avro_dat = writer.into_inner().unwrap();
            avro_dat
        }

        pub fn from_avro(avro_dat: Vec<u8>) -> Locked {
            use apache_avro::{from_value, Reader};
            let reader = Reader::new(&avro_dat[..]).unwrap();
            let db = from_value::<AvroLocked>(&reader.last().unwrap().unwrap()).unwrap();
            db.to_locked_struct()
        }

        pub fn to_vec(&self) -> Vec<u8> {
            to_ecc(self.to_avro())
        }

        pub fn from_vec(dat_ecc: Vec<u8>) -> Locked {
            Locked::from_avro(from_ecc(dat_ecc))
        }

    }

}

In [4]:
let mut db = Database::Unlocked::new("test_key");
db

Unlocked { db: {}, meta: {"app": "digisafe", "timestamp": "1699588749", "version": "1.0.0", "uid": "Kb+kH90y8DyIX+9fnEnt208re4fCNbKOcfZaB1jGOFM=", "revision": "0"}, key: [146, 251, 216, 195, 51, 10, 173, 46, 196, 91, 169, 181, 149, 93, 179, 66, 192, 91, 240, 175, 185, 222, 31, 183, 198, 99, 75, 154, 88, 8, 146, 245] }

In [5]:
db.set_private("name", "main");
db

Unlocked { db: {"_name": "main"}, meta: {"app": "digisafe", "timestamp": "1699588749", "version": "1.0.0", "uid": "Kb+kH90y8DyIX+9fnEnt208re4fCNbKOcfZaB1jGOFM=", "revision": "0"}, key: [146, 251, 216, 195, 51, 10, 173, 46, 196, 91, 169, 181, 149, 93, 179, 66, 192, 91, 240, 175, 185, 222, 31, 183, 198, 99, 75, 154, 88, 8, 146, 245] }

In [6]:
db.set("k1", "v1");
db.set("k2", "v2");
db.set("k3", "v3");
db

Unlocked { db: {"k3": "v3", "k1": "v1", "k2": "v2", "_name": "main"}, meta: {"app": "digisafe", "timestamp": "1699588749", "version": "1.0.0", "uid": "Kb+kH90y8DyIX+9fnEnt208re4fCNbKOcfZaB1jGOFM=", "revision": "0"}, key: [146, 251, 216, 195, 51, 10, 173, 46, 196, 91, 169, 181, 149, 93, 179, 66, 192, 91, 240, 175, 185, 222, 31, 183, 198, 99, 75, 154, 88, 8, 146, 245] }

In [7]:
let db = db.lock();
db

Locked { db: "K1kC1kzitnSI9cIR/iOmjNltnfDug5YKRBgVNdTLAUCfVTQu0AVXTl7Z1MKmSOMpu4R1dfpIMZRTEWeCroqQiO+SQLGKkNfQutcrj4naXljJ/nyrs1jMlctAQE/6S3r7XfyCW4t3+erQgRHOBscxBZ8MQefbFZg5N9lufiTI7uBbygcOdRkOJ2TD/SogMcI2Rgv6p9qvkvOQ10DkaKEaHH0n4kbOTghIEM0Ls2h7YMZ5COUzzeycvCMMibVHRBznxU/fGsw75IDQzDXwAvnR96VU9YJ/Vyc4aPOVSuTjlZNJSsvBIHpL/vCv3M+E4c+5/wuMb9+1UMhb06rEx8SX4xErErFLbv7DoXfuAr1g25lgPGlRPxbOUCKVdJ4RtTLEcduf44JO6WPQm5WEDroaaJ0i/2VDTg4rw8YeEZDhg5Hh3WBDoLNAUbMYrTzW2gm1", meta: {"app": "digisafe", "timestamp": "1699588749", "uid": "Kb+kH90y8DyIX+9fnEnt208re4fCNbKOcfZaB1jGOFM=", "version": "1.0.0", "nonce": "FyenU52UdbCcoWyNYS6f5dC9e1EGF3Cz", "revision": "0"}, key: [146, 251, 216, 195, 51, 10, 173, 46, 196, 91, 169, 181, 149, 93, 179, 66, 192, 91, 240, 175, 185, 222, 31, 183, 198, 99, 75, 154, 88, 8, 146, 245] }

In [8]:
let db = db.unlock("test_key").unwrap();
db

Unlocked { db: {"k3": "v3", "k1": "v1", "_name": "main", "k2": "v2"}, meta: {"app": "digisafe", "revision": "0", "timestamp": "1699588749", "uid": "Kb+kH90y8DyIX+9fnEnt208re4fCNbKOcfZaB1jGOFM=", "version": "1.0.0"}, key: [146, 251, 216, 195, 51, 10, 173, 46, 196, 91, 169, 181, 149, 93, 179, 66, 192, 91, 240, 175, 185, 222, 31, 183, 198, 99, 75, 154, 88, 8, 146, 245] }

In [9]:
let db = db.lock();
db

Locked { db: "OZ6B+GZeZxeb4jEshF8Vp5dEulSKKfoFcaBdYPpyke5HMKE1Uvc+uCiav0EN3jQMS+pTU7iC+bLQxrzIjNpoycQjd3uz2WEhIzxoBPxATH8a9W4lKC9tbb0Fw7hmavYgONp+8MC3vjAbydrBk/mEH1PLzDAF0/BLVck8C5WU+QbmjeP/YqAvtoIEIx6nQDFTSUHIukH9bZ0YFYUdVXh+gChMkBdYacXnXaTccvdoil/seTbTQjpvUJAdlhgP2MiRqQnHFCgDfioCeQYbIopbEUSAoJ+1I0FV+6jwVd6+990n6RVnMqHmBq8yHOLn5a87Cncc3hpbWTFBDegFrVHwkEro7g37XJAffkGLZ0RNuGcK+H6QDNhvlLMBO7z1GyFTglbKKZ4c+TH3h5ARuP8EDX3zpiZ2AyxBpykQ8tkp/GzluekLPy5CdaW+KBR+SA==", meta: {"nonce": "PUE5vvz/X09Tfr5yBHtDSfkxb/54Km21", "revision": "0", "timestamp": "1699588749", "app": "digisafe", "uid": "Kb+kH90y8DyIX+9fnEnt208re4fCNbKOcfZaB1jGOFM=", "version": "1.0.0"}, key: [146, 251, 216, 195, 51, 10, 173, 46, 196, 91, 169, 181, 149, 93, 179, 66, 192, 91, 240, 175, 185, 222, 31, 183, 198, 99, 75, 154, 88, 8, 146, 245] }

In [10]:
let mut db_vec = db.to_vec();
db_vec

[79, 98, 106, 1, 4, 22, 97, 118, 114, 111, 46, 115, 99, 104, 101, 109, 97, 132, 2, 123, 34, 116, 121, 112, 101, 34, 58, 34, 114, 101, 99, 111, 114, 100, 34, 44, 34, 110, 97, 109, 101, 34, 58, 34, 108, 111, 99, 107, 101, 100, 34, 44, 34, 102, 105, 101, 108, 100, 115, 34, 58, 91, 123, 34, 110, 97, 109, 101, 34, 58, 34, 100, 98, 34, 44, 34, 116, 121, 112, 101, 34, 58, 34, 115, 116, 114, 105, 110, 103, 34, 125, 44, 123, 34, 110, 97, 109, 101, 34, 58, 34, 109, 101, 116, 97, 34, 44, 34, 116, 121, 112, 101, 34, 58, 123, 34, 116, 121, 112, 101, 34, 58, 34, 109, 97, 112, 34, 44, 34, 118, 97, 108, 117, 101, 115, 34, 58, 34, 115, 116, 114, 105, 110, 103, 34, 125, 125, 93, 125, 20, 97, 118, 114, 111, 46, 99, 111, 100, 101, 99, 8, 110, 117, 108, 108, 0, 210, 35, 94, 229, 182, 108, 161, 220, 106, 178, 218, 227, 26, 8, 41, 74, 2, 174, 9, 128, 7, 79, 90, 54, 66, 43, 71, 90, 101, 90, 120, 101, 98, 52, 106, 69, 115, 104, 70, 56, 86, 112, 53, 100, 69, 117, 108, 83, 75, 75, 102, 111, 70, 99, 97, 66, 100, 

In [11]:
db_vec[0] = 0;
db_vec[10] = 0;
db_vec[100] = 0;
db_vec[200] = 0;
db_vec

[0, 98, 106, 1, 4, 22, 97, 118, 114, 111, 0, 115, 99, 104, 101, 109, 97, 132, 2, 123, 34, 116, 121, 112, 101, 34, 58, 34, 114, 101, 99, 111, 114, 100, 34, 44, 34, 110, 97, 109, 101, 34, 58, 34, 108, 111, 99, 107, 101, 100, 34, 44, 34, 102, 105, 101, 108, 100, 115, 34, 58, 91, 123, 34, 110, 97, 109, 101, 34, 58, 34, 100, 98, 34, 44, 34, 116, 121, 112, 101, 34, 58, 34, 115, 116, 114, 105, 110, 103, 34, 125, 44, 123, 34, 110, 97, 109, 101, 34, 58, 0, 109, 101, 116, 97, 34, 44, 34, 116, 121, 112, 101, 34, 58, 123, 34, 116, 121, 112, 101, 34, 58, 34, 109, 97, 112, 34, 44, 34, 118, 97, 108, 117, 101, 115, 34, 58, 34, 115, 116, 114, 105, 110, 103, 34, 125, 125, 93, 125, 20, 97, 118, 114, 111, 46, 99, 111, 100, 101, 99, 8, 110, 117, 108, 108, 0, 210, 35, 94, 229, 182, 108, 161, 220, 106, 178, 218, 227, 26, 8, 41, 74, 2, 174, 9, 128, 7, 79, 90, 54, 66, 43, 71, 90, 101, 90, 120, 101, 98, 52, 0, 69, 115, 104, 70, 56, 86, 112, 53, 100, 69, 117, 108, 83, 75, 75, 102, 111, 70, 99, 97, 66, 100, 89, 8

In [12]:
let mut db = Database::Locked::from_vec(db_vec);
db

Locked { db: "OZ6B+GZeZxeb4jEshF8Vp5dEulSKKfoFcaBdYPpyke5HMKE1Uvc+uCiav0EN3jQMS+pTU7iC+bLQxrzIjNpoycQjd3uz2WEhIzxoBPxATH8a9W4lKC9tbb0Fw7hmavYgONp+8MC3vjAbydrBk/mEH1PLzDAF0/BLVck8C5WU+QbmjeP/YqAvtoIEIx6nQDFTSUHIukH9bZ0YFYUdVXh+gChMkBdYacXnXaTccvdoil/seTbTQjpvUJAdlhgP2MiRqQnHFCgDfioCeQYbIopbEUSAoJ+1I0FV+6jwVd6+990n6RVnMqHmBq8yHOLn5a87Cncc3hpbWTFBDegFrVHwkEro7g37XJAffkGLZ0RNuGcK+H6QDNhvlLMBO7z1GyFTglbKKZ4c+TH3h5ARuP8EDX3zpiZ2AyxBpykQ8tkp/GzluekLPy5CdaW+KBR+SA==", meta: {"uid": "Kb+kH90y8DyIX+9fnEnt208re4fCNbKOcfZaB1jGOFM=", "version": "1.0.0", "revision": "0", "nonce": "PUE5vvz/X09Tfr5yBHtDSfkxb/54Km21", "timestamp": "1699588749", "app": "digisafe"}, key: [] }

In [13]:
use std::io::{Read, Write};
let file_name = "test.digisafe";
let mut fout = std::fs::File::create(file_name).unwrap();
fout.write_all(&db.to_vec()[..]).unwrap();
std::fs::read(file_name).unwrap()

[79, 98, 106, 1, 4, 22, 97, 118, 114, 111, 46, 115, 99, 104, 101, 109, 97, 132, 2, 123, 34, 116, 121, 112, 101, 34, 58, 34, 114, 101, 99, 111, 114, 100, 34, 44, 34, 110, 97, 109, 101, 34, 58, 34, 108, 111, 99, 107, 101, 100, 34, 44, 34, 102, 105, 101, 108, 100, 115, 34, 58, 91, 123, 34, 110, 97, 109, 101, 34, 58, 34, 100, 98, 34, 44, 34, 116, 121, 112, 101, 34, 58, 34, 115, 116, 114, 105, 110, 103, 34, 125, 44, 123, 34, 110, 97, 109, 101, 34, 58, 34, 109, 101, 116, 97, 34, 44, 34, 116, 121, 112, 101, 34, 58, 123, 34, 116, 121, 112, 101, 34, 58, 34, 109, 97, 112, 34, 44, 34, 118, 97, 108, 117, 101, 115, 34, 58, 34, 115, 116, 114, 105, 110, 103, 34, 125, 125, 93, 125, 20, 97, 118, 114, 111, 46, 99, 111, 100, 101, 99, 8, 110, 117, 108, 108, 0, 154, 31, 163, 44, 152, 119, 195, 236, 109, 69, 138, 67, 169, 8, 24, 71, 2, 174, 9, 128, 7, 79, 90, 54, 66, 43, 71, 90, 101, 90, 120, 101, 98, 52, 106, 69, 115, 104, 70, 56, 86, 112, 53, 100, 69, 117, 108, 83, 75, 75, 102, 111, 70, 99, 97, 66, 100, 8